In [ ]:
#Import the data from dat files. 
import numpy as np
training_data= []
for line in open('data/train_pr2.dat','r'):
    training_data.append(line)
print (len(training_data))

testing_data= []
for line in open('data/test_pr2.dat','r'):
    testing_data.append(line)
print (len(testing_data))
 
        
 

###### Seperate the class file from training data. 

In [ ]:
#print (training_data[1:2])
target =[]
training_data_raw = []
for i in training_data:
    target.append(i[0])
    training_data_raw.append(i[1:])
testing_data_raw = []
for i in testing_data:
    testing_data_raw.append(i[1:]) 
#print(testing_data_raw[1])
target[1:4]

###### Append the training and testing data into single total list for csr matrix

In [ ]:
print(training_data_raw[0]) 

total = training_data_raw + testing_data_raw
print (len(total))

###### Split the lines into words i.e doc(list of lists)

In [ ]:
docs = [l.split() for l in total_raw]
print(len(docs))

In [ ]:
#code for computation of CSR Matrix
from collections import Counter
from scipy.sparse import csr_matrix
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat


def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

###### Build CSR matrix for total doc 

In [ ]:
mat = build_matrix(docs)
csr_info(mat)
mat

In [ ]:
#split the training and test data
train_mat = mat[0:800]
print (train_mat.shape)
test_mat = mat[800:]
print (test_mat.shape)

###### Reduce the Dimensionality of the Data 

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=800)
svd.fit(train_mat)
train_svd = svd.transform(train_mat)
print(type(train_svd))

test_svd = svd.transform(test_mat)



In [ ]:
#print the shape of training and test matrix after dimensionality reduction
print (train_svd.shape)
print (test_svd.shape)
print(len(target))

In [ ]:
#apply the Adaboost classifier to classify the data
from sklearn.ensemble import AdaBoostClassifier
adaBoost = AdaBoostClassifier(n_estimators=1000, learning_rate=1.5)
adaBoost.fit(train_svd,target)


In [ ]:
#obtain the result onto the variable result
result = adaBoost.predict(test_svd)

In [ ]:
#write the result onto the file
f=open("result3.txt", "w")
for i in range(0,350):
    f.write(result[i])
    f.write('\n')
